In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import grad

In [2]:
device = torch.device("cuda")

torch.manual_seed(2023)
np.random.seed(2023)

from scipy.integrate import odeint

In [6]:
def SIR(net, t, params):
    N, beta, gamma = params
    u = net.forward(t)
    S = u[:, 0:1]
    I = u[:, 1:2]
    R = u[:, 2:3]
    S_t = grad(S.sum(), t, create_graph=True)[0]
    I_t = grad(I.sum(), t, create_graph=True)[0]
    R_t = grad(R.sum(), t, create_graph=True)[0]
    u_t = torch.cat([S_t, I_t, R_t], dim=1)
    loss_f = u_t + torch.cat([beta * S * I/N, -beta * S * I/N + gamma * I, -gamma * I], dim=1)
    loss_f = torch.mean(torch.square(loss_f))
    return loss_f

In [7]:
# ODE parameters
N = 100        # population
beta = 1.0     # infected person infects 1 other person per day
D = 4.0        # infections lasts D days
gamma = 1.0 / D
SIR.params = [N, beta, gamma]

# Initial condition
S0 = 99
I0 = 1
R0 = 0

# time domain
tl = 0 
tr = 20

In [8]:
def deriv(u, t, N, beta, gamma):
    S, I, R = u
    dSdt = -beta * S * I/N
    dIdt = beta * S * I/N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [10]:
t = np.linspace(tl, tr, 501)
u0 = S0, I0, R0
u = odeint(deriv, u0, t, args=(N, beta, gamma))

X_test = torch.tensor(t, dtype = torch.float64, device = device).view(-1, 1)
U_test = torch.tensor(u, dtype = torch.float64, device = device).view(-1, 3)

N_colloc = 2000

X_colloc = torch.rand(N_colloc, dtype = torch.float64, device = device).view(-1, 1) * (tr - tl) + tl
X_colloc.requires_grad_()

x_mean = torch.mean(X_colloc, 0).detach() # detach()를 통해 X_colloc의 gradient를 계산하지 않음
X_std = torch.std(X_colloc, 0).detach()

X_ic = torch.tensor([0], dtype = torch.float64, device = device).view(-1, 1)
U_ic = torch.tensor([S0, I0, R0], dtype = torch.float64, device = device).view(-1, 3)

U_mean = torch.mean(U_ic)
U_std = torch.std(U_ic)

def calculate_relative_err(net):
    U_pred = net.predict(X_test)
    err = torch.abs(U_pred - U_test)
    err = torch.linalg.norm(err)/torch.linalg.norm(U_test)
    return err.item()



In [18]:
x = torch.tensor(1.0, requires_grad=True)

y = 2 * x




In [19]:
z = y.detach()
y.backward()

In [20]:
print(x.grad)
print(z.grad)

tensor(2.)
None


In [21]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc(x)
        return x

model = SimpleNet()
inputs = torch.tensor([[1.0, 2.0]])
outputs = model(inputs)
print(outputs)


tensor([[1.1558]], grad_fn=<AddmmBackward0>)
